# PubMedBERTを用いたテキスト分類

このノートブックでは、PubMedBertを用いて、テキスト分類をする方法を紹介します。PubMedBERTは大規模な医療系テキストを用いて学習されたモデルです。医療系のタスクにおいては、Wikipediaやニュース記事で学習したBERTよりも高性能となることが期待できます。

訳注: 元のノートブックではBioBERTを利用していましたが、PubMedBERTの論文を見る限り、こちらのほうが性能が良かったので置き換えています。

- [Domain-Specific Language Model Pretraining for Biomedical Natural Language Processing](https://arxiv.org/abs/2007.15779)


## 準備

### パッケージのインストール

In [1]:
!pip install -q pandas tensorflow transformers scikit-learn

     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 7.5 MB/s 
     |████████████████████████████████| 67 kB 2.3 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 


### インポート

In [86]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification
tf.get_logger().setLevel('ERROR')

### データのダウンロードとアップロード

今回のデータセットは、Kaggleの以下のページからダウンロードする必要があります。登録してダウンロードし、`mtsamples.csv`をアップロードしましょう。

- [Medical Transcriptions](https://www.kaggle.com/tboyle10/medicaltranscriptions)

In [3]:
from google.colab import files

uploaded = files.upload()

Saving mtsamples.csv to mtsamples.csv


### データの読み込み

In [102]:
df = pd.read_csv("mtsamples.csv")
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [103]:
df["medical_specialty"].value_counts()

 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               

このデータセットを使って、`transcription`から`medical_speciality`を予測するモデルを学習しましょう。データセットは非常に不均衡なので、数の少ないクラスを除去したり、まとめたり、オーバー/アンダーサンプリングするのも1つの手です。今回はそのまま使います。

## 前処理

`LabelEncoder`を使って、ラベルの文字列を数字に変換します。

In [104]:
le = LabelEncoder()
df["medical_specialty"] = le.fit_transform(df["medical_specialty"])
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,0,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,2,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,2,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,3,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,3,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [105]:
df.dropna(subset=['transcription'], inplace=True)

データセットを学習用とテスト用に分割します。

In [106]:
x_train, x_test, y_train, y_test = train_test_split(
    list(df["transcription"].values),
    list(df["medical_specialty"].values),
    test_size=0.2,
    random_state=2021
)

x_valid, x_test, y_valid, y_test = train_test_split(
    x_test,
    y_test,
    test_size=0.5,
    random_state=2021
)

In [107]:
x_train[0]

'PREOPERATIVE DIAGNOSIS: , Extremely large basal cell carcinoma, right lower lid.,POSTOPERATIVE DIAGNOSIS:,  Extremely large basal cell carcinoma, right lower lid.,TITLE OF OPERATION: , Excision of large basal cell carcinoma, right lower lid, and repaired with used dorsal conjunctival flap in the upper lid and a large preauricular skin graft.,PROCEDURE: , The patient was brought into the operating room and prepped and draped in usual fashion.  Xylocaine 2% with epinephrine was injected beneath the conjunctiva and skin of the lower lid and also beneath the conjunctiva and skin of the upper lid.  A frontal nerve block was also given on the right upper lid.  The anesthetic agent was also injected in the right preauricular region which would provide a donor graft for the right lower lid defect.  The area was marked with a marking pen with margins of 3 to 4 mm, and a #15 Bard-Parker blade was used to make an incision at the nasal and temporal margins of the lesion.,The incision was carried 

`BERTTokenizerFast.from_pretrained`メソッドでトークナイザーをインスタンス化します。

In [108]:
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

トークナイザーにテキストを与えて、エンコーディングしましょう。

In [109]:
MAX_LEN = 128
train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LEN)
val_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LEN)
test_encodings = tokenizer(x_test, truncation=True, padding="max_length",  max_length=MAX_LEN)

`tf.data.Dataset`の`from_tensor_slices`メソッドにラベルとエンコーディングした入力を与えて、データセットを作成しましょう。

In [110]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_valid
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [111]:
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(8)
val_dataset = val_dataset.batch(8)
test_dataset = test_dataset.batch(8)

## モデルの学習

`TFAutoModelForSequenceClassification`を使って分類用のモデルを作成したら、Kerasのfitメソッドを呼び出して学習しましょう。

In [112]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(le.classes_),
    from_pt=True
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=2,
        restore_best_weights=True
    ),
]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_dataset, validation_data=val_dataset, epochs=5, callbacks=callbacks)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
497/497 [==============================] - 141s 257ms/step - loss: 2.2700 - sparse_categorical_accuracy: 0.3419 - val_loss: 1.8805 - val_sparse_categorical_accuracy: 0.3803
Epoch 2/5
497/497 [==============================] - 127s 255ms/step - loss: 1.7071 - sparse_categorical_accuracy: 0.3885 - val_loss: 1.7262 - val_sparse_categorical_accuracy: 0.4024
Epoch 3/5
497/497 [==============================] - 127s 256ms/step - loss: 1.5147 - sparse_categorical_accuracy: 0.4139 - val_loss: 1.7612 - val_sparse_categorical_accuracy: 0.3843
Epoch 4/5
497/497 [==============================] - 127s 255ms/step - loss: 1.4046 - sparse_categorical_accuracy: 0.4215 - val_loss: 1.7338 - val_sparse_categorical_accuracy: 0.3179


正解率を計算してみましょう。

In [113]:
_, acc = model.evaluate(test_dataset)
acc

63/63 [==============================] - 5s 82ms/step - loss: 1.7514 - sparse_categorical_accuracy: 0.3682


0.36820924282073975